In [339]:
import pandas as pd

df_happ = pd.read_csv("drive/My Drive/FERI/MAG/TRIR/Data/happiness_oecd.csv", sep=';', decimal=',', index_col='Country')
df_happ.shape

(168, 40)

In [340]:
df_happ.loc[df_happ['Region'] == 'Australia and New Zealand', 'continent'] = 'Other'
df_happ.loc[df_happ['Region'].isin(['Central and Eastern Europe', 'Western Europe']), 'continent'] = 'Europe'
df_happ.loc[df_happ['Region'].isin(['Eastern Asia', 'Southeastern Asia', 'Southern Asia']), 'continent'] = 'Asia'
df_happ.loc[df_happ['Region'].isin(['Latin America and Caribbean', 'North America']), 'continent'] = 'Americas'
df_happ.loc[df_happ['Region'].isin(['Middle East and Northern Africa', 'Sub-Saharan Africa']), 'continent'] = 'Africa'
df_happ.head()

,Happiness.Rank,Happiness.Score,Economy..GDP.per.Capita.,Family,Health..Life.Expectancy.,Freedom,Generosity,Trust..Government.Corruption.,Dystopia.Residual,Region,Happiness Rank 2016,Happiness Score 2016,Happiness Rank 2015,Happiness Score 2015,Air pollution (Micrograms per cubic metre),Dwellings without basic facilities (Percentage),Educational attainment (Percentage),Employees working very long hours (Percentage),Employment rate (Percentage),Feeling safe walking alone at night (Percentage),Homicide rate (Ratio),Household net adjusted disposable income (US Dollar),Household net financial wealth (US Dollar),Housing expenditure (Percentage),Labour market insecurity (Percentage),Life expectancy (Years),Life satisfaction (Average score),Long-term unemployment rate (Percentage),Personal earnings (US Dollar),Quality of support network (Percentage),Rooms per person (Ratio),Self-reported health (Percentage),Stakeholder engagement for developing regulations (Average score),Student skills (Average score),Time devoted to leisure and personal care (Hours),Voter turnout (Percentage),Water quality (Percentage),Years in education (Years),RankDiff2017,RankDiff2016,continent
Country,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Afghanistan,141.0,3.79399991035461,0.401477217674255,0.581543326377869,0.18074677884578694,0.106179520487785,0.311870932579041,0.0611578300595284,2.15080118179321,Southern Asia,154.0,3.36,153.0,3.575,13.394736842105264,3.4499999999999997,77.23684210526316,8.723157894736842,67.73684210526316,68.63421052631578,2.934210526315789,25113.842105263157,49362.78947368421,20.894736842105264,5.476315789473684,79.53947368421052,6.528947368421054,3.1976315789473677,37435.94736842105,90.02631578947368,1.6421052631578947,67.44736842105263,2.05,486.7631578947368,14.834210526315792,70.02631578947368,82.26315789473684,17.376315789473683,-13.0,1.0,Asia
Albania,109.0,4.64400005340576,0.996192753314972,0.8036852478981021,0.731159746646881,0.381498634815216,0.201312944293022,0.0398642159998417,1.49044156074524,Central and Eastern Europe,109.0,4.655,95.0,4.959,13.394736842105264,3.4499999999999997,77.23684210526316,8.723157894736842,67.73684210526316,68.63421052631578,2.934210526315789,25113.842105263157,49362.78947368421,20.894736842105264,5.476315789473684,79.53947368421052,6.528947368421054,3.1976315789473677,37435.94736842105,90.02631578947368,1.6421052631578947,67.44736842105263,2.05,486.7631578947368,14.834210526315792,70.02631578947368,82.26315789473684,17.376315789473683,0.0,14.0,Europe
Algeria,53.0,5.8720002174377415,1.09186446666718,1.1462174654007,0.6175846457481379,0.233335807919502,0.0694366469979286,0.14609611034393302,2.5676038265228303,Middle East and Northern Africa,38.0,6.355,68.0,5.605,13.394736842105264,3.4499999999999997,77.23684210526316,8.723157894736842,67.73684210526316,68.63421052631578,2.934210526315789,25113.842105263157,49362.78947368421,20.894736842105264,5.476315789473684,79.53947368421052,6.528947368421054,3.1976315789473677,37435.94736842105,90.02631578947368,1.6421052631578947,67.44736842105263,2.05,486.7631578947368,14.834210526315792,70.02631578947368,82.26315789473684,17.376315789473683,15.0,-30.0,Africa
Angola,140.0,3.79500007629395,0.8584281802177429,1.10441195964813,0.0498686656355858,0.0,0.09792649000883097,0.0697203353047371,1.61448240280151,Sub-Saharan Africa,141.0,3.866,137.0,4.033,13.394736842105264,3.4499999999999997,77.23684210526316,8.723157894736842,67.73684210526316,68.63421052631578,2.934210526315789,25113.842105263157,49362.78947368421,20.894736842105264,5.476315789473684,79.53947368421052,6.528947368421054,3.1976315789473677,37435.94736842105,90.02631578947368,1.6421052631578947,67.44736842105263,2.05,486.7631578947368,14.834210526315792,70.02631578947368,82.26315789473684,17.376315789473683,-1.0,4.0,Africa
Argentina,24.0,6.59899997711182,1.18529546260834,1.44045114517212,0.695137083530426,0.4945192039012911,0.109457060694695,0.059739887714385986,2.61400532722473,Latin America and Caribbean,26.

In [341]:
df_world = pd.read_csv("drive/My Drive/FERI/MAG/TRIR/Data/worldbook.csv", sep=',', decimal='.', index_col='Country')
df_world.shape

(229, 12)

In [342]:
df = pd.concat([df_happ, df_world], axis=1, sort=True)
df.shape

(246, 53)

In [343]:
df.head()

,Happiness.Rank,Happiness.Score,Economy..GDP.per.Capita.,Family,Health..Life.Expectancy.,Freedom,Generosity,Trust..Government.Corruption.,Dystopia.Residual,Region,Happiness Rank 2016,Happiness Score 2016,Happiness Rank 2015,Happiness Score 2015,Air pollution (Micrograms per cubic metre),Dwellings without basic facilities (Percentage),Educational attainment (Percentage),Employees working very long hours (Percentage),Employment rate (Percentage),Feeling safe walking alone at night (Percentage),Homicide rate (Ratio),Household net adjusted disposable income (US Dollar),Household net financial wealth (US Dollar),Housing expenditure (Percentage),Labour market insecurity (Percentage),Life expectancy (Years),Life satisfaction (Average score),Long-term unemployment rate (Percentage),Personal earnings (US Dollar),Quality of support network (Percentage),Rooms per person (Ratio),Self-reported health (Percentage),Stakeholder engagement for developing regulations (Average score),Student skills (Average score),Time devoted to leisure and personal care (Hours),Voter turnout (Percentage),Water quality (Percentage),Years in education (Years),RankDiff2017,RankDiff2016,continent,Background,Mean elavation,Agricultural land use (%),Total dependency ratio,Median age (total),Population growth rate,Birth rate,Death rate,Net migration rate,Urban population (%),Rate of urbanization (%),Sex ratio (total)
Afghanistan,141.0,3.79399991035461,0.401477217674255,0.581543326377869,0.18074677884578694,0.106179520487785,0.311870932579041,0.0611578300595284,2.15080118179321,Southern Asia,154.0,3.36,153.0,3.575,13.394736842105264,3.4499999999999997,77.23684210526316,8.723157894736842,67.73684210526316,68.63421052631578,2.934210526315789,25113.842105263157,49362.78947368421,20.894736842105264,5.476315789473684,79.53947368421052,6.528947368421054,3.1976315789473677,37435.94736842105,90.02631578947368,1.6421052631578947,67.44736842105263,2.05,486.7631578947368,14.834210526315792,70.02631578947368,82.26315789473684,17.376315789473683,-13.0,1.0,Asia,Ahmad Shah DURRANI unified the Pashtun tribes ...,1.884,58.1,88.8,19.5,2.38,36.7,12.7,-0.1,26.0,3.37,102.60
Albania,109.0,4.64400005340576,0.996192753314972,0.8036852478981021,0.731159746646881,0.381498634815216,0.201312944293022,0.0398642159998417,1.49044156074524,Central and Eastern Europe,109.0,4.655,95.0,4.959,13.394736842105264,3.4499999999999997,77.23684210526316,8.723157894736842,67.73684210526316,68.63421052631578,2.934210526315789,25113.842105263157,49362.78947368421,20.894736842105264,5.476315789473684,79.53947368421052,6.528947368421054,3.1976315789473677,37435.94736842105,90.02631578947368,1.6421052631578947,67.44736842105263,2.05,486.7631578947368,14.834210526315792,70.02631578947368,82.26315789473684,17.376315789473683,0.0,14.0,Europe,Albania declared its independence from the Ott...,708.000,43.1,46.9,34.3,0.28,13.0,7.1,-3.3,62.1,1.69,0.98
Algeria,53.0,5.8720002174377415,1.09186446666718,1.1462174654007,0.6175846457481379,0.233335807919502,0.0694366469979286,0.14609611034393302,2.5676038265228303,Middle East and Northern Africa,38.0,6.355,68.0,5.605,13.394736842105264,3.4499999999999997,77.23684210526316,8.723157894736842,67.73684210526316,68.63421052631578,2.934210526315789,25113.842105263157,49362.78947368421,20.894736842105264,5.476315789473684,79.53947368421052,6.528947368421054,3.1976315789473677,37435.94736842105,90.02631578947368,1.6421052631578947,67.44736842105263,2.05,486.7631578947368,14.834210526315792,70.02631578947368,82.26315789473684,17.376315789473683,15.0,-30.0,Africa,"After more than a century of rule by France, A...",800.000,17.4,60.1,28.9,1.52,20.0,4.4,-0.9,73.7,2.46,102.70
American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Settled as early as 1000 B.C., Samoa was not r...",NaN,21.9,NaN,27.2,-1.40,17.8,5.9,-26.1,87.2,0.07,99.60
Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [344]:
df.drop(['Background', 'Region'], axis=1, inplace=True)
df.head()

,Happiness.Rank,Happiness.Score,Economy..GDP.per.Capita.,Family,Health..Life.Expectancy.,Freedom,Generosity,Trust..Government.Corruption.,Dystopia.Residual,Happiness Rank 2016,Happiness Score 2016,Happiness Rank 2015,Happiness Score 2015,Air pollution (Micrograms per cubic metre),Dwellings without basic facilities (Percentage),Educational attainment (Percentage),Employees working very long hours (Percentage),Employment rate (Percentage),Feeling safe walking alone at night (Percentage),Homicide rate (Ratio),Household net adjusted disposable income (US Dollar),Household net financial wealth (US Dollar),Housing expenditure (Percentage),Labour market insecurity (Percentage),Life expectancy (Years),Life satisfaction (Average score),Long-term unemployment rate (Percentage),Personal earnings (US Dollar),Quality of support network (Percentage),Rooms per person (Ratio),Self-reported health (Percentage),Stakeholder engagement for developing regulations (Average score),Student skills (Average score),Time devoted to leisure and personal care (Hours),Voter turnout (Percentage),Water quality (Percentage),Years in education (Years),RankDiff2017,RankDiff2016,continent,Mean elavation,Agricultural land use (%),Total dependency ratio,Median age (total),Population growth rate,Birth rate,Death rate,Net migration rate,Urban population (%),Rate of urbanization (%),Sex ratio (total)
Afghanistan,141.0,3.79399991035461,0.401477217674255,0.581543326377869,0.18074677884578694,0.106179520487785,0.311870932579041,0.0611578300595284,2.15080118179321,154.0,3.36,153.0,3.575,13.394736842105264,3.4499999999999997,77.23684210526316,8.723157894736842,67.73684210526316,68.63421052631578,2.934210526315789,25113.842105263157,49362.78947368421,20.894736842105264,5.476315789473684,79.53947368421052,6.528947368421054,3.1976315789473677,37435.94736842105,90.02631578947368,1.6421052631578947,67.44736842105263,2.05,486.7631578947368,14.834210526315792,70.02631578947368,82.26315789473684,17.376315789473683,-13.0,1.0,Asia,1.884,58.1,88.8,19.5,2.38,36.7,12.7,-0.1,26.0,3.37,102.60
Albania,109.0,4.64400005340576,0.996192753314972,0.8036852478981021,0.731159746646881,0.381498634815216,0.201312944293022,0.0398642159998417,1.49044156074524,109.0,4.655,95.0,4.959,13.394736842105264,3.4499999999999997,77.23684210526316,8.723157894736842,67.73684210526316,68.63421052631578,2.934210526315789,25113.842105263157,49362.78947368421,20.894736842105264,5.476315789473684,79.53947368421052,6.528947368421054,3.1976315789473677,37435.94736842105,90.02631578947368,1.6421052631578947,67.44736842105263,2.05,486.7631578947368,14.834210526315792,70.02631578947368,82.26315789473684,17.376315789473683,0.0,14.0,Europe,708.000,43.1,46.9,34.3,0.28,13.0,7.1,-3.3,62.1,1.69,0.98
Algeria,53.0,5.8720002174377415,1.09186446666718,1.1462174654007,0.6175846457481379,0.233335807919502,0.0694366469979286,0.14609611034393302,2.5676038265228303,38.0,6.355,68.0,5.605,13.394736842105264,3.4499999999999997,77.23684210526316,8.723157894736842,67.73684210526316,68.63421052631578,2.934210526315789,25113.842105263157,49362.78947368421,20.894736842105264,5.476315789473684,79.53947368421052,6.528947368421054,3.1976315789473677,37435.94736842105,90.02631578947368,1.6421052631578947,67.44736842105263,2.05,486.7631578947368,14.834210526315792,70.02631578947368,82.26315789473684,17.376315789473683,15.0,-30.0,Africa,800.000,17.4,60.1,28.9,1.52,20.0,4.4,-0.9,73.7,2.46,102.70
American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.9,NaN,27.2,-1.40,17.8,5.9,-26.1,87.2,0.07,99.60
Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.996,40.0,NaN,46.2,-0.06,7.0,7.7,0.0,87.9,-0.31,105.50


In [345]:
df['continent'].isna().sum()

78

In [346]:
from sklearn_pandas import CategoricalImputer
from sklearn.preprocessing import LabelEncoder

df['continent'] = CategoricalImputer().fit_transform(df['continent'])
df['continent'] = LabelEncoder().fit_transform(df['continent'])
df.head(10)

,Happiness.Rank,Happiness.Score,Economy..GDP.per.Capita.,Family,Health..Life.Expectancy.,Freedom,Generosity,Trust..Government.Corruption.,Dystopia.Residual,Happiness Rank 2016,Happiness Score 2016,Happiness Rank 2015,Happiness Score 2015,Air pollution (Micrograms per cubic metre),Dwellings without basic facilities (Percentage),Educational attainment (Percentage),Employees working very long hours (Percentage),Employment rate (Percentage),Feeling safe walking alone at night (Percentage),Homicide rate (Ratio),Household net adjusted disposable income (US Dollar),Household net financial wealth (US Dollar),Housing expenditure (Percentage),Labour market insecurity (Percentage),Life expectancy (Years),Life satisfaction (Average score),Long-term unemployment rate (Percentage),Personal earnings (US Dollar),Quality of support network (Percentage),Rooms per person (Ratio),Self-reported health (Percentage),Stakeholder engagement for developing regulations (Average score),Student skills (Average score),Time devoted to leisure and personal care (Hours),Voter turnout (Percentage),Water quality (Percentage),Years in education (Years),RankDiff2017,RankDiff2016,continent,Mean elavation,Agricultural land use (%),Total dependency ratio,Median age (total),Population growth rate,Birth rate,Death rate,Net migration rate,Urban population (%),Rate of urbanization (%),Sex ratio (total)
Afghanistan,141.0,3.79399991035461,0.401477217674255,0.581543326377869,0.18074677884578694,0.106179520487785,0.311870932579041,0.0611578300595284,2.15080118179321,154.0,3.36,153.0,3.575,13.394736842105264,3.4499999999999997,77.23684210526316,8.723157894736842,67.73684210526316,68.63421052631578,2.934210526315789,25113.842105263157,49362.78947368421,20.894736842105264,5.476315789473684,79.53947368421052,6.528947368421054,3.1976315789473677,37435.94736842105,90.02631578947368,1.6421052631578947,67.44736842105263,2.05,486.7631578947368,14.834210526315792,70.02631578947368,82.26315789473684,17.376315789473683,-13.0,1.0,2,1.884,58.1,88.8,19.5,2.38,36.7,12.7,-0.1,26.0,3.37,102.60
Albania,109.0,4.64400005340576,0.996192753314972,0.8036852478981021,0.731159746646881,0.381498634815216,0.201312944293022,0.0398642159998417,1.49044156074524,109.0,4.655,95.0,4.959,13.394736842105264,3.4499999999999997,77.23684210526316,8.723157894736842,67.73684210526316,68.63421052631578,2.934210526315789,25113.842105263157,49362.78947368421,20.894736842105264,5.476315789473684,79.53947368421052,6.528947368421054,3.1976315789473677,37435.94736842105,90.02631578947368,1.6421052631578947,67.44736842105263,2.05,486.7631578947368,14.834210526315792,70.02631578947368,82.26315789473684,17.376315789473683,0.0,14.0,3,708.000,43.1,46.9,34.3,0.28,13.0,7.1,-3.3,62.1,1.69,0.98
Algeria,53.0,5.8720002174377415,1.09186446666718,1.1462174654007,0.6175846457481379,0.233335807919502,0.0694366469979286,0.14609611034393302,2.5676038265228303,38.0,6.355,68.0,5.605,13.394736842105264,3.4499999999999997,77.23684210526316,8.723157894736842,67.73684210526316,68.63421052631578,2.934210526315789,25113.842105263157,49362.78947368421,20.894736842105264,5.476315789473684,79.53947368421052,6.528947368421054,3.1976315789473677,37435.94736842105,90.02631578947368,1.6421052631578947,67.44736842105263,2.05,486.7631578947368,14.834210526315792,70.02631578947368,82.26315789473684,17.376315789473683,15.0,-30.0,0,800.000,17.4,60.1,28.9,1.52,20.0,4.4,-0.9,73.7,2.46,102.70
American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,21.9,NaN,27.2,-1.40,17.8,5.9,-26.1,87.2,0.07,99.60
Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1.996,40.0,NaN,46.2,-0.06,7.0,7.7,0.0,87.9,-0.31,105.50
Angola,140.0,3.79500007629395,0.8584281802177429,1.10441195964813,0.0498686656355858,0.0,0.09792649000883097,0.0697203353047371,

In [347]:
from sklearn.impute import SimpleImputer

index = np.where(df.columns=='continent')
num_cols = np.delete(df.columns, index)

mean_values = SimpleImputer(missing_values=np.nan, strategy='mean')

df[num_cols] = mean_values.fit_transform(df[num_cols])
df.head()

,Happiness.Rank,Happiness.Score,Economy..GDP.per.Capita.,Family,Health..Life.Expectancy.,Freedom,Generosity,Trust..Government.Corruption.,Dystopia.Residual,Happiness Rank 2016,Happiness Score 2016,Happiness Rank 2015,Happiness Score 2015,Air pollution (Micrograms per cubic metre),Dwellings without basic facilities (Percentage),Educational attainment (Percentage),Employees working very long hours (Percentage),Employment rate (Percentage),Feeling safe walking alone at night (Percentage),Homicide rate (Ratio),Household net adjusted disposable income (US Dollar),Household net financial wealth (US Dollar),Housing expenditure (Percentage),Labour market insecurity (Percentage),Life expectancy (Years),Life satisfaction (Average score),Long-term unemployment rate (Percentage),Personal earnings (US Dollar),Quality of support network (Percentage),Rooms per person (Ratio),Self-reported health (Percentage),Stakeholder engagement for developing regulations (Average score),Student skills (Average score),Time devoted to leisure and personal care (Hours),Voter turnout (Percentage),Water quality (Percentage),Years in education (Years),RankDiff2017,RankDiff2016,continent,Mean elavation,Agricultural land use (%),Total dependency ratio,Median age (total),Population growth rate,Birth rate,Death rate,Net migration rate,Urban population (%),Rate of urbanization (%),Sex ratio (total)
Afghanistan,141.0,3.794000,0.401477,0.581543,0.180747,0.106180,0.311871,0.061158,2.150801,154.000000,3.360000,153.000000,3.575000,13.394737,3.45,77.236842,8.723158,67.736842,68.634211,2.934211,25113.842105,49362.789474,20.894737,5.476316,79.539474,6.528947,3.197632,37435.947368,90.026316,1.642105,67.447368,2.05,486.763158,14.834211,70.026316,82.263158,17.376316,-13.000000,1.000000,2,1.884000,58.1,88.800000,19.5,2.38,36.7,12.7,-0.1,26.0,3.37,102.60
Albania,109.0,4.644000,0.996193,0.803685,0.731160,0.381499,0.201313,0.039864,1.490442,109.000000,4.655000,95.000000,4.959000,13.394737,3.45,77.236842,8.723158,67.736842,68.634211,2.934211,25113.842105,49362.789474,20.894737,5.476316,79.539474,6.528947,3.197632,37435.947368,90.026316,1.642105,67.447368,2.05,486.763158,14.834211,70.026316,82.263158,17.376316,0.000000,14.000000,3,708.000000,43.1,46.900000,34.3,0.28,13.0,7.1,-3.3,62.1,1.69,0.98
Algeria,53.0,5.872000,1.091864,1.146217,0.617585,0.233336,0.069437,0.146096,2.567604,38.000000,6.355000,68.000000,5.605000,13.394737,3.45,77.236842,8.723158,67.736842,68.634211,2.934211,25113.842105,49362.789474,20.894737,5.476316,79.539474,6.528947,3.197632,37435.947368,90.026316,1.642105,67.447368,2.05,486.763158,14.834211,70.026316,82.263158,17.376316,15.000000,-30.000000,0,800.000000,17.4,60.100000,28.9,1.52,20.0,4.4,-0.9,73.7,2.46,102.70
American Samoa,78.0,5.354019,0.984718,1.188898,0.551341,0.408786,0.246883,0.123120,1.850238,78.980892,5.382185,79.493671,5.375734,13.394737,3.45,77.236842,8.723158,67.736842,68.634211,2.934211,25113.842105,49362.789474,20.894737,5.476316,79.539474,6.528947,3.197632,37435.947368,90.026316,1.642105,67.447368,2.05,486.763158,14.834211,70.026316,82.263158,17.376316,-0.980892,-0.512779,0,311.827657,21.9,58.742424,27.2,-1.40,17.8,5.9,-26.1,87.2,0.07,99.60
Andorra,78.0,5.354019,0.984718,1.188898,0.551341,0.408786,0.246883,0.123120,1.850238,78.980892,5.382185,79.493671,5.375734,13.394737,3.45,77.236842,8.723158,67.736842,68.634211,2.934211,25113.842105,49362.789474,20.894737,5.476316,79.539474,6.528947,3.197632,37435.947368,90.026316,1.642105,67.447368,2.05,486.763158,14.834211,70.026316,82.263158,17.376316,-0.980892,-0.512779,0,1.996000,40.0,58.742424,46.2,-0.06,7.0,7.7,0.0,87.9,-0.31,105.50


In [348]:
'''from sklearn.preprocessing import StandardScaler

df[num_cols] = StandardScaler().fit_transform(df[num_cols])
df.head()'''

'from sklearn.preprocessing import StandardScaler\n\ndf[num_cols] = StandardScaler().fit_transform(df[num_cols])\ndf.head()'

In [0]:
from sklearn.model_selection import train_test_split

input_cols = num_cols
output_cols = 'continent'

x_train, x_test, y_train, y_test = train_test_split(df[input_cols], df[output_cols], test_size=0.25, train_size=0.75)

In [0]:
from sklearn.ensemble import RandomForestClassifier
import warnings

with warnings.catch_warnings():
  warnings.simplefilter('ignore')

  rfc = RandomForestClassifier(n_estimators=500)
  rfc.fit(x_train, y_train)
  rfc_res = rfc.predict(x_test)
  rfc_res

In [351]:
from sklearn.metrics import accuracy_score, f1_score

acc_score = accuracy_score(y_test, rfc_res)
f1_score = f1_score(y_test, rfc_res, average='micro')

print(f'Accuracy score: {acc_score}')
print(f'F1 score: {f1_score}')

Accuracy score: 0.7903225806451613
F1 score: 0.7903225806451614
